In [25]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import scanpy as sc
import anndata
import torch
import numpy as np 
import pandas as pd
import torch.nn.functional as F
from torch import nn
from torch import optim
from sklearn.model_selection import KFold, StratifiedKFold
from torch.utils.data import DataLoader,TensorDataset,SubsetRandomSampler, WeightedRandomSampler

sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=80, frameon=False, figsize=(3, 3), facecolor='white')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
humdata = np.load('5 x_hum_0902_anno_nn.npz', allow_pickle=True)
humnmf = pd.read_pickle('6 Ws_hum_0902')
x_hum = torch.from_numpy(humnmf[11]).float() # 1,2,5,11
y_hum = torch.from_numpy(humdata['y'].astype(np.int32))
y_hum = y_hum.long() - 1

dataset = TensorDataset(x_hum, y_hum)

inputSize = x_hum.shape[1]
outputSize = 14

num_epochs = 30
batch_size = 8
lr = 0.02

sample_size = 8
sample_idx=[]
for t in np.unique(y_hum):
    t_idx = np.where(y_hum==t)[0]
    sample_idx.append(np.random.choice(t_idx, size=sample_size))
sample_idx = np.vstack(sample_idx)
sample_idx = sample_idx.reshape(sample_idx.size)
x_train = x_hum[sample_idx]
y_train = y_hum[sample_idx]

dataset = TensorDataset(x_train, y_train)

In [36]:
class MLR(nn.Module):
    def __init__(self):
        super(MLR, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(inputSize, outputSize)
        )
            
    def forward(self, x):
        out = self.layers(x)
        if not self.training:
            #print("TESTING MODE")
            out = F.softmax(out, dim=1)
        #out = out.view(out.shape[0], -1)
        return out

In [37]:
device = "cuda:0"
torch.manual_seed(42)

loss_fn = nn.CrossEntropyLoss()

def train(model, device, dataloader, loss_fn, optimizer):
    train_loss, train_correct=0.0,0
    model.train()

    for inputs, labels in dataloader:
        inputs,labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(inputs.float())

        loss = loss_fn(output,labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * inputs.size(0)
        scores, predictions = torch.max(output.data, 1)
        train_correct += (predictions == labels).sum().item()

    return train_loss,train_correct

def val(model, device, dataloader, loss_fn):
    valid_loss, val_correct = 0.0, 0
    model.eval()

    for inputs, labels in dataloader:
        inputs,labels = inputs.to(device),labels.to(device)
        output = model(inputs)
        loss = loss_fn(output,labels)
        valid_loss += loss.item()*inputs.size(0)
        scores, predictions = torch.max(output.data,1)
        val_correct += (predictions == labels).sum().item()

    return valid_loss,val_correct


train_loader = DataLoader(dataset, batch_size=batch_size)
test_loader = DataLoader(dataset, batch_size=batch_size)

model = torch.load('MLR_NMF_r60.pt')
model.to(device)
optimizer = optim.RMSprop(model.parameters(), lr=lr)

for epoch in range(num_epochs):
    train_loss, train_correct = train(model, device, train_loader, loss_fn, optimizer)
    test_loss, test_correct = val(model, device, test_loader, loss_fn)

    train_loss = train_loss / len(train_loader.sampler)
    train_acc = train_correct / len(train_loader.sampler) * 100
    test_loss = test_loss / len(test_loader.sampler)
    test_acc = test_correct / len(test_loader.sampler) * 100

    print("Epoch:{}/{} AVG Training Loss:{:.3f} AVG Test Loss:{:.3f} AVG Training Acc {:.2f} % AVG Test Acc {:.2f} %".format(
        epoch + 1, num_epochs, train_loss, test_loss, train_acc, test_acc))

D:\Miniconda3\lib\site-packages\torch\serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
D:\Miniconda3\lib\site-packages\torch\serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


Epoch:1/30 AVG Training Loss:3.106 AVG Test Loss:2.573 AVG Training Acc 0.00 % AVG Test Acc 19.32 %
Epoch:2/30 AVG Training Loss:2.363 AVG Test Loss:2.532 AVG Training Acc 14.77 % AVG Test Acc 32.95 %
Epoch:3/30 AVG Training Loss:2.031 AVG Test Loss:2.496 AVG Training Acc 25.00 % AVG Test Acc 45.45 %
Epoch:4/30 AVG Training Loss:1.805 AVG Test Loss:2.466 AVG Training Acc 40.91 % AVG Test Acc 55.68 %
Epoch:5/30 AVG Training Loss:1.635 AVG Test Loss:2.438 AVG Training Acc 51.14 % AVG Test Acc 64.77 %
Epoch:6/30 AVG Training Loss:1.500 AVG Test Loss:2.411 AVG Training Acc 62.50 % AVG Test Acc 80.68 %
Epoch:7/30 AVG Training Loss:1.391 AVG Test Loss:2.387 AVG Training Acc 76.14 % AVG Test Acc 86.36 %
Epoch:8/30 AVG Training Loss:1.299 AVG Test Loss:2.365 AVG Training Acc 80.68 % AVG Test Acc 90.91 %
Epoch:9/30 AVG Training Loss:1.220 AVG Test Loss:2.344 AVG Training Acc 88.64 % AVG Test Acc 96.59 %
Epoch:10/30 AVG Training Loss:1.150 AVG Test Loss:2.324 AVG Training Acc 92.05 % AVG Test Ac

In [38]:
torch.save(model, 'MLR_NMF_r60_retrain.pt')